In [50]:
import psycopg2
import requests

In [38]:
conn=psycopg2.connect(database="stockgraphdb",
                 user="aryanpatel",
                 password="12345",
                 host="localhost",
                 port="5432")

cursor=conn.cursor()

In [39]:
sqlQuery="""INSERT INTO adanient.adanient_1m (open,close,high,low,volume,epochtime) VALUES(2.00, 4.0, 3.5 ,4.5 ,2244 ,1682513967)
ON CONFLICT(epochtime)  
DO UPDATE SET
open = EXCLUDED.open,
close=EXCLUDED.close,
high=EXCLUDED.high,
volume=EXCLUDED.volume;
"""


try:
    cursor.execute(sqlQuery)
    conn.commit()
except Exception as e:
    print("Error:", e)
    conn.rollback()
finally:
    cursor.close()
    conn.close()

In [54]:
import pytz
from datetime import datetime, timedelta
def generate_groww_api_url(symbol, start_date_str, end_date_str, interval_minutes):
    ist = pytz.timezone("Asia/Kolkata")
    
    # Convert string dates to datetime
    start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
    end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
    
    # Set time to 9:15 AM for start date
    start_datetime = ist.localize(datetime.combine(start_date, datetime.strptime("09:15", "%H:%M").time()))
    
    # Set time to 3:30 PM for end date
    end_datetime = ist.localize(datetime.combine(end_date, datetime.strptime("15:30", "%H:%M").time()))
    
    # Convert to milliseconds
    start_millis = int(start_datetime.timestamp() * 1000)
    end_millis = int(end_datetime.timestamp() * 1000)
    
    url = f"https://groww.in/v1/api/charting_service/v4/chart/exchange/NSE/segment/CASH/{symbol}?intervalInMinutes={interval_minutes}&startTimeInMillis={start_millis}&endTimeInMillis={end_millis}"
    
    return url

# Example usage:

symbol = "ADANIENT"
start_date = "2024-05-08"   # 8th May 2024
end_date = "2025-04-25"     # 25th April 2025
interval = 1                # 1 minute

url = generate_groww_api_url(symbol, start_date, end_date, interval)
print(url)

https://groww.in/v1/api/charting_service/v4/chart/exchange/NSE/segment/CASH/ADANIENT?intervalInMinutes=1&startTimeInMillis=1715139900000&endTimeInMillis=1745575200000


In [ ]:
def generate_intraday_intervals(end_date, interval_name):
    ist = pytz.timezone("Asia/Kolkata")

    # Fixed start date: Wednesday, 8 May 2024, 09:15 AM IST
    fixed_start_date = datetime(2024, 5, 8, 9, 15)
    fixed_start_datetime = ist.localize(fixed_start_date)

    # Move end date 7 days back
    end_date_adjusted = end_date - timedelta(days=7)

    # Ensure only weekdays and proper market timing
    intervals = []

    current_date = fixed_start_datetime.date()
    final_date = end_date_adjusted.date()

    interval_minutes = interval_map.get(interval_name, 1)  # default 1 min if not found

    while current_date <= final_date:
        # Only weekdays (Mon-Fri)
        if current_date.weekday() < 5:
            start_datetime = ist.localize(datetime.combine(current_date, datetime.strptime("09:15", "%H:%M").time()))
            end_datetime = ist.localize(datetime.combine(current_date, datetime.strptime("15:30", "%H:%M").time()))

            intervals.append((start_datetime, end_datetime))

        current_date += timedelta(days=1)

    return intervals





In [55]:
start_date = datetime(2025 , 1, 1) # Example start date    
#create intervals every week after 1 week and end week is today

today = datetime.now()
##end date should be startdate + 7 days until today
intervals = []

while start_date < today:
    intervals.append((start_date, start_date + timedelta(days=7)))
    start_date = start_date + timedelta(days=7)

In [56]:
intervals

[(datetime.datetime(2025, 1, 1, 0, 0), datetime.datetime(2025, 1, 8, 0, 0)),
 (datetime.datetime(2025, 1, 8, 0, 0), datetime.datetime(2025, 1, 15, 0, 0)),
 (datetime.datetime(2025, 1, 15, 0, 0), datetime.datetime(2025, 1, 22, 0, 0)),
 (datetime.datetime(2025, 1, 22, 0, 0), datetime.datetime(2025, 1, 29, 0, 0)),
 (datetime.datetime(2025, 1, 29, 0, 0), datetime.datetime(2025, 2, 5, 0, 0)),
 (datetime.datetime(2025, 2, 5, 0, 0), datetime.datetime(2025, 2, 12, 0, 0)),
 (datetime.datetime(2025, 2, 12, 0, 0), datetime.datetime(2025, 2, 19, 0, 0)),
 (datetime.datetime(2025, 2, 19, 0, 0), datetime.datetime(2025, 2, 26, 0, 0)),
 (datetime.datetime(2025, 2, 26, 0, 0), datetime.datetime(2025, 3, 5, 0, 0)),
 (datetime.datetime(2025, 3, 5, 0, 0), datetime.datetime(2025, 3, 12, 0, 0)),
 (datetime.datetime(2025, 3, 12, 0, 0), datetime.datetime(2025, 3, 19, 0, 0)),
 (datetime.datetime(2025, 3, 19, 0, 0), datetime.datetime(2025, 3, 26, 0, 0)),
 (datetime.datetime(2025, 3, 26, 0, 0), datetime.datetime(2

In [59]:
#convert intervasl to epochime
def convert_to_epoch(intervals):
    epoch_intervals = []
    for start, end in intervals:
        start_epoch = int(start.timestamp())
        end_epoch = int(end.timestamp())
        epoch_intervals.append((start_epoch, end_epoch))
    return epoch_intervals

In [61]:
convert_to_epoch(intervals)

[(1735669800, 1736274600),
 (1736274600, 1736879400),
 (1736879400, 1737484200),
 (1737484200, 1738089000),
 (1738089000, 1738693800),
 (1738693800, 1739298600),
 (1739298600, 1739903400),
 (1739903400, 1740508200),
 (1740508200, 1741113000),
 (1741113000, 1741717800),
 (1741717800, 1742322600),
 (1742322600, 1742927400),
 (1742927400, 1743532200),
 (1743532200, 1744137000),
 (1744137000, 1744741800),
 (1744741800, 1745346600),
 (1745346600, 1745951400)]

In [74]:
def connection():
    conn = psycopg2.connect(
        database="stockanalysis",
        user='aryanpatel',
        password="12345",
        host="localhost",
        port="5432"
    )
    return conn

def create_table(conn, stock_name, interval_name):
    
    conn.cursor().execute(f'CREATE SCHEMA IF NOT EXISTS {stock_name}')  # Use quotes to handle special characters

    conn.cursor().execute(f"""
        CREATE TABLE IF NOT EXISTS {stock_name}.candle_{interval_name} (
            id SERIAL PRIMARY KEY,
            open FLOAT,
            high FLOAT,
            low FLOAT,
            close FLOAT,
            volume FLOAT,
            epochtime BIGINT UNIQUE NOT NULL
        )
    """)
    conn.commit()
    

In [76]:
conn = connection()
stock_name = "M&M"
interval_name = "1m"
create_table(conn, stock_name, interval_name)
conn.close()

SyntaxError: syntax error at or near "&"
LINE 1: CREATE SCHEMA IF NOT EXISTS M&M
                                     ^
